# Multicriterio


In [2]:
import numpy 
import pandas as pd

datos=pd.read_excel(r"C:\Users\marti\OneDrive\Documentos\personal\Universidad\Septimo_semestre\Opti_multiobjective\Multicriterio\AHP_multicriterio.xlsx")
datos.head()

,Unnamed: 0,Unnamed: 1,Max,Max.1,Min,Max.2,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,NaN,Cell,RAM,Gb,Precio,Cámara,Sist,Color,RAM,Gb,Precio,Cámara,Sist,Color,Rank
1,NaN,A32,4,64,0.8,20,Android,Morado,0.2,0.096774,0.857143,0.2,1,0.666667,0.46159
2,NaN,A03,4,32,0.65,15,Android,Negro,0.2,0.032258,0.964286,0.057143,1,1,0.472306
3,NaN,13promax,8,512,2,48,Ios,Celeste,1,1,0,1,0.5,0.333333,0.634921
4,NaN,Redmi9,3,32,0.6,13,Android,Rojo,0,0.032258,1,0,1,0,0.388633
